# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 9 2022 8:30AM,245999,10,bps200075,Beach Patient Assistance,Released
1,Sep 9 2022 8:30AM,246000,10,SOTAH0000773,"Nextsource Biotechnology, LLC",Released
2,Sep 8 2022 3:35PM,245998,12,HH09082022,Hush Hush,Released
3,Sep 8 2022 3:34PM,245997,18,INNOVA-A9Z62B,Innova Softgel LLC,Released
4,Sep 8 2022 2:25PM,245990,10,8495841,Beach Products Inc,Released
5,Sep 8 2022 2:25PM,245990,10,8495843,Beach Products Inc,Released
6,Sep 8 2022 2:25PM,245989,10,MSP213796,"Methapharm, Inc.",Released
7,Sep 8 2022 2:25PM,245989,10,MSP213797,"Methapharm, Inc.",Executing
8,Sep 8 2022 2:25PM,245989,10,MSP213798,"Methapharm, Inc.",Executing
9,Sep 8 2022 2:25PM,245989,10,MSP213799,"Methapharm, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,245990,Released,2
25,245997,Released,1
26,245998,Released,1
27,245999,Released,1
28,246000,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245990,NaN,NaN,NaN,2.0
245997,NaN,NaN,NaN,1.0
245998,NaN,NaN,NaN,1.0
245999,NaN,NaN,NaN,1.0
246000,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245758,1.0,0.0,0.0,0.0
245898,0.0,0.0,0.0,4.0
245916,0.0,0.0,0.0,1.0
245917,0.0,1.0,2.0,16.0
245925,0.0,1.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245758,1.0,0,0,0
245898,0.0,0,0,4
245916,0.0,0,0,1
245917,0.0,1,2,16
245925,0.0,1,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245758,1.0,0,0,0
1,245898,0.0,0,0,4
2,245916,0.0,0,0,1
3,245917,0.0,1,2,16
4,245925,0.0,1,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245758,1.0,,,
1,245898,0.0,,,4
2,245916,0.0,,,1
3,245917,0.0,1,2,16
4,245925,0.0,1,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance
1,Sep 9 2022 8:30AM,246000,10,"Nextsource Biotechnology, LLC"
2,Sep 8 2022 3:35PM,245998,12,Hush Hush
3,Sep 8 2022 3:34PM,245997,18,Innova Softgel LLC
4,Sep 8 2022 2:25PM,245990,10,Beach Products Inc
6,Sep 8 2022 2:25PM,245989,10,"Methapharm, Inc."
16,Sep 8 2022 2:22PM,245988,16,"SHL Pharma, LLC"
17,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC"
18,Sep 8 2022 1:47PM,245982,10,ISDIN Corporation
30,Sep 8 2022 1:37PM,245981,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,0.0,,,1
1,Sep 9 2022 8:30AM,246000,10,"Nextsource Biotechnology, LLC",0.0,,,1
2,Sep 8 2022 3:35PM,245998,12,Hush Hush,0.0,,,1
3,Sep 8 2022 3:34PM,245997,18,Innova Softgel LLC,0.0,,,1
4,Sep 8 2022 2:25PM,245990,10,Beach Products Inc,0.0,,,2
5,Sep 8 2022 2:25PM,245989,10,"Methapharm, Inc.",0.0,,9,1
6,Sep 8 2022 2:22PM,245988,16,"SHL Pharma, LLC",0.0,,1,
7,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC",0.0,,,1
8,Sep 8 2022 1:47PM,245982,10,ISDIN Corporation,0.0,,,12
9,Sep 8 2022 1:37PM,245981,10,"Senseonics, Incorporated",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1,,
1,Sep 9 2022 8:30AM,246000,10,"Nextsource Biotechnology, LLC",1,,
2,Sep 8 2022 3:35PM,245998,12,Hush Hush,1,,
3,Sep 8 2022 3:34PM,245997,18,Innova Softgel LLC,1,,
4,Sep 8 2022 2:25PM,245990,10,Beach Products Inc,2,,
5,Sep 8 2022 2:25PM,245989,10,"Methapharm, Inc.",1,9,
6,Sep 8 2022 2:22PM,245988,16,"SHL Pharma, LLC",,1,
7,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC",1,,
8,Sep 8 2022 1:47PM,245982,10,ISDIN Corporation,12,,
9,Sep 8 2022 1:37PM,245981,10,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1,,
1,Sep 9 2022 8:30AM,246000,10,"Nextsource Biotechnology, LLC",1,,
2,Sep 8 2022 3:35PM,245998,12,Hush Hush,1,,
3,Sep 8 2022 3:34PM,245997,18,Innova Softgel LLC,1,,
4,Sep 8 2022 2:25PM,245990,10,Beach Products Inc,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1.0,NaN,NaN
1,Sep 9 2022 8:30AM,246000,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Sep 8 2022 3:35PM,245998,12,Hush Hush,1.0,NaN,NaN
3,Sep 8 2022 3:34PM,245997,18,Innova Softgel LLC,1.0,NaN,NaN
4,Sep 8 2022 2:25PM,245990,10,Beach Products Inc,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1.0,0.0,0.0
1,Sep 9 2022 8:30AM,246000,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Sep 8 2022 3:35PM,245998,12,Hush Hush,1.0,0.0,0.0
3,Sep 8 2022 3:34PM,245997,18,Innova Softgel LLC,1.0,0.0,0.0
4,Sep 8 2022 2:25PM,245990,10,Beach Products Inc,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2705433,50.0,12.0,2.0
12,245998,1.0,0.0,0.0
15,491910,48.0,40.0,15.0
16,983844,6.0,1.0,0.0
18,491931,1.0,1.0,0.0
19,245916,1.0,0.0,0.0
22,245967,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2705433,50.0,12.0,2.0
1,12,245998,1.0,0.0,0.0
2,15,491910,48.0,40.0,15.0
3,16,983844,6.0,1.0,0.0
4,18,491931,1.0,1.0,0.0
5,19,245916,1.0,0.0,0.0
6,22,245967,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,50.0,12.0,2.0
1,12,1.0,0.0,0.0
2,15,48.0,40.0,15.0
3,16,6.0,1.0,0.0
4,18,1.0,1.0,0.0
5,19,1.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,50.0
1,12,Released,1.0
2,15,Released,48.0
3,16,Released,6.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,22
Status,,,,,,,
Completed,2.0,0.0,15.0,0.0,0.0,0.0,0.0
Executing,12.0,0.0,40.0,1.0,1.0,0.0,0.0
Released,50.0,1.0,48.0,6.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,22
0,Completed,2.0,0.0,15.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,40.0,1.0,1.0,0.0,0.0
2,Released,50.0,1.0,48.0,6.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,22
0,Completed,2.0,0.0,15.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,40.0,1.0,1.0,0.0,0.0
2,Released,50.0,1.0,48.0,6.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()